# Conjunto de Dados 1 e 2: *Montgomery e Shenzhen*
***
* Disponível em: <https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities>. Acesso em 19 fev. 2021.
* Disponível em: <https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities>. Acesso em 19 fev. 2021.

### Importação dos pacotes

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tqdm import tqdm
import urllib.request
import pandas as pd
import numpy as np
import glob
import re

### Pré-processamento dos Dados

In [3]:
file1 = '../0-datasets/montgomery-shenzen/arquivos-descompactados/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/*.png'
filelist_shenzen = glob.glob(file1)
file2 = '../0-datasets/montgomery-shenzen/arquivos-descompactados/Montgomery/MontgomerySet/CXR_png/*.png'
filelist_montgomery = glob.glob(file2)
filelist = filelist_shenzen + filelist_montgomery

print('quantidade de imagens:', str(len(filelist)))

quantidade de imagens: 800


In [4]:
def extract_label(file_list):
  '''
  extraindo os rótulos da imagem

  Args:
    file_list (list) --> lista com a referência dos arquivos de imagem
  
  Returns:
    labels (list) --> lista com as rotulações da imagem
  '''
    
  labels = []
    
  for file in tqdm(file_list): 
    current_label = re.findall('[0-9]{4}_(.+?).png', file) 
    if current_label[0] == '0':
      labels.append('normal')
    else:
      labels.append('abnormal')
        
  return labels

In [6]:
labels = extract_label(filelist)
print('quantidade de rótulos:', str(len(labels)))

quantidade de rótulos: 800


In [11]:
full_data = pd.DataFrame(filelist, columns = ['filepath'])
full_data['target'] = labels
train_df, test_df = train_test_split(full_data, stratify = full_data['target'], test_size = 0.2, random_state = 42)
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'], test_size = 0.2, random_state = 42)

print('quantidade de imagens de treinamento:', len(train_df['filepath']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['filepath']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['filepath']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 512
quantidade de rótulos de treinamento: 512
quantidade de imagens de teste: 160
quantidade de rótulos de teste: 160
quantidade de imagens de validação: 128
quantidade de rótulos de validação: 128


In [13]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['target']), train_df['target'])
class_weight = {0: class_weights[0], 1: class_weights[1]}

train_df.to_csv('train_df.csv')
validation_df.to_csv('validation_df.csv')
test_df.to_csv('test_df.csv')

### Gerador de dados para a rede pelo Keras

In [14]:
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True,
                                     rotation_range = 20, zoom_range = 0.2)

train_generator = image_generator.flow_from_dataframe(dataframe = train_df, directory = '', x_col = 'filepath',
                                                      y_col = 'target', batch_size = 32, seed = 42,
                                                      shuffle = True, class_mode = 'categorical',
                                                      target_size = (256, 256))
 
valid_generator = image_generator.flow_from_dataframe(dataframe = validation_df, directory = '.', x_col = 'filepath',
                                                      y_col = 'target', batch_size = 32, seed = 42,
                                                      shuffle = True, class_mode = 'categorical',
                                                      target_size = (256, 256))

test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_df, directory = '.', x_col = 'filepath',
                                                  y_col = 'target', batch_size = 32, seed = 42,
                                                  shuffle = True, class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 512 validated image filenames belonging to 2 classes.
Found 128 validated image filenames belonging to 2 classes.
Found 160 validated image filenames belonging to 2 classes.


In [15]:
train_generator.class_indices

{'abnormal': 0, 'normal': 1}